The [`httpfs`](https://duckdb.org/docs/extensions/httpfs/s3api, "httpfs") extension supports reading/writing/globbing files on object storage servers using the S3 API. S3 offers a standard API to read and write to remote files (while regular http servers, predating S3, do not offer a common write API). DuckDB conforms to the S3 API, that is now common among industry storage providers.

The preferred way to configure and authenticate to S3 endpoints is to use secrets. Multiple secret providers are available

# ETL

```yaml metadata
name: S3_EXTRACT
description: "Example extrating from S3 to a local sqlite3 file"
connection: "duckdb:"
active: true
```

## train_services

```yaml metadata
name: train_services
description: "train_services"
table: train_services
load_conn: "duckdb:"
load_before_sql:
  - load_extentions
  - attach_db
load_sql: load_query
load_after_sql: detach_db
drop_sql: drop_sql
clean_sql: clean_sql
rows_sql: nrows
active: true
```

```sql
-- load_extentions
INSTALL Sqlite;
LOAD Sqlite;
INSTALL httpfs;
LOAD httpfs;
```

```sql
-- attach_db
ATTACH 'examples/S3_EXTRACT.db' AS "DB" (TYPE SQLITE)
```

```sql
-- detach_db
DETACH "DB";
```

```sql
-- load_query
CREATE OR REPLACE TABLE "DB"."<table>" AS
FROM 's3://duckdb-blobs/train_services.parquet';
```

```sql
-- drop_sql
DROP TABLE IF EXISTS "DB"."<table>";
```

```sql
-- clean_sql
DELETE FROM "DB"."<table>";
```

```sql
-- nrows
SELECT COUNT(*) AS "nrows" FROM "DB"."<table>"
```

## S3_EXTRACT

```yaml metadata
name: S3_EXTRACT
description: "Example extrating from S3 to a local sqlite3 file"
table: S3_EXTRACT
load_conn: "duckdb:"
load_before_sql:
  - load_extentions
  - attach_db
  - create_S3_token
load_sql: load_query
load_after_sql: detach_db
drop_sql: drop_sql
clean_sql: clean_sql
rows_sql: nrows
active: true
```

```sql
-- load_extentions
INSTALL httpfs;
LOAD httpfs;
```

```sql
-- attach_db
ATTACH 'examples/S3_EXTRACT.db' AS "DB" (TYPE SQLITE)
```

Example with a [Minio](https://min.io/) local instance

```sql
-- create_S3_token
CREATE SECRET S3_token (
   TYPE S3,
   KEY_ID '@S3_KEY_ID',
   SECRET '@S3_SECRET',
   ENDPOINT '127.0.0.1:3000',
   URL_STYLE 'path'
);
```

```sql
-- detach_db
DETACH "DB";
```

```sql
-- load_query
CREATE OR REPLACE TABLE "DB"."<table>" AS
SELECT * 
FROM 's3://uploads/flights.csv';
```

```sql
-- drop_sql
DROP TABLE IF EXISTS "DB"."<table>";
```

```sql
-- clean_sql
DELETE FROM "DB"."<table>";
```

```sql
-- nrows
SELECT COUNT(*) AS "nrows" FROM "DB"."<table>"
```

```shell
bin/etlx --config examples/s3.ipynb
```

```shell
bin/etlx --config examples/s3.ipynb
```